In [1]:
from transformers import BertModel, DistilBertModel
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
import numpy as np
import pickle as pkl
import os
from os.path import join as oj
from spacy.lang.en import English
import argparse
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
from datasets import load_from_disk
import sklearn
import warnings

We're going to aggregate embeddings run for a particular order of n-gram here (we'll just add them up).

In [2]:
processed_dir = 'data/processed'
dir_names = [f for f in sorted(os.listdir(processed_dir))
             if not '-all' in f
            ]

In [ ]:
for i in tqdm(range(len(dir_names))):
    s = dir_names[i]
    start = s.index('=') + 1
    end = s.index('_')
    num = int(s[ start: end])
    if num > 1:
        print('Trying for', s)
        s_new = s + '-all'
        if os.path.exists(oj(processed_dir, s_new)):
            print('\tdone already!')
            continue
        pre = s[:start]
        end = s[end:]
        reloaded_dataset = load_from_disk(oj(processed_dir, s))
        X_train = np.array(reloaded_dataset['train']['embs']).squeeze()
        X_val = np.array(reloaded_dataset['validation']['embs']).squeeze()
        num_smalls = []
        for num_small in range(1, num):
            fname_small = pre + str(num_small) + end
            if fname_small in dir_names:
                reloaded_dataset = load_from_disk(oj(processed_dir, fname_small))
                X_train_small = np.array(reloaded_dataset['train']['embs']).squeeze()
                X_val_small = np.array(reloaded_dataset['validation']['embs']).squeeze()
                num_smalls.append(num_small)
                
                X_train += X_train_small
                X_val += X_val_small
        if num_smalls == list(range(1, num)):
            os.makedirs(oj(processed_dir, s_new), exist_ok=True)
            mu = X_train.mean(axis=0)
            sigma = X_train.std(axis=0)
            r = {
                'X_train': X_train,
                'X_val': X_val,
                'mean': mu,
                'sigma': sigma,
            }
            pkl.dump(r, open(oj(processed_dir, s_new, 'data.pkl'), 'wb'))
            print('\tsuccess!')
        else:
            print('\tfailed!', num_smalls)

In [45]:
X_train.shape

(100, 768)

In [48]:
mean = 

In [51]:
(X_train - mean) / X_train.std(axis=0)

array([[ 0.39971717,  0.47728783,  0.46848973, ...,  0.32394024,
         0.40770636, -0.41445175],
       [ 0.37845816,  0.27007233, -0.6737011 , ..., -0.7822047 ,
         0.34070138, -0.36811865],
       [-0.28291333, -0.26231812, -1.51527908, ..., -1.70423174,
        -0.27811537,  0.23633958],
       ...,
       [ 0.59832025,  0.59225591,  0.1777626 , ...,  0.29190491,
         0.60562303, -0.59645348],
       [ 0.72135488,  0.72851596,  0.32479591, ...,  0.17907882,
         0.70840679, -0.71539313],
       [-0.94252311, -0.62276438, -0.20464616, ...,  0.7566754 ,
        -0.82595086,  0.91553885]])